<a href="https://colab.research.google.com/github/HWANG-HOE-SUN/Univ.Project/blob/main/%EC%83%81%EA%B6%8C%EC%83%81%EC%83%9D/%EC%97%85%EC%A2%85_%EC%97%B0%EA%B4%80%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/') 

Mounted at /content/gdrive/


In [208]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime
import time
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
folder = '/content/gdrive/MyDrive/빅분경공유폴더/'

In [5]:
file_list = os.listdir(folder)
file_list

['2019년 소비자행태조사 원시데이터_문항,보기.xlsx',
 '2019년 소비자행태조사 원시데이터_답변.csv',
 '다매체 시대, 한국의 소비자는 누구일까  한국의 소비자 유형과 광고매체 효과 비교.pdf',
 '★2020+방송매체이용행태조사_최종.pdf',
 '2020방송매체이용행태조사_개인_공표용.SAV',
 '서울시 우리마을가게 상권분석서비스(상권-점포).csv',
 '서울시 우리마을가게 상권분석서비스(상권배후지-소득소비).csv',
 '중간결과_상권군집별_업종별_1분기변화%.xlsx',
 '마포구_행정동별_63업종별_1th금액계차이의증감율(%).xlsx',
 '마포구_행정동별_63업종별_5th금액계차이의증감율(%).xlsx',
 '마포구_상권별_업종별_개수.xlsx']

In [6]:
file_name = []
for file in file_list:
  name = folder+'/'+file
  file_name.append(name)

In [212]:
class 분기변화율_스케일조정:
  def __init__(self,분기,data):
    from sklearn.preprocessing import MinMaxScaler
    self.scaler = MinMaxScaler()
    self.df = data
    self.분기 = 분기
    self.temp = self.df['{}분기_변화%'.format(self.분기)] 
    if __name__ == "__main__":
      print("자동으로 해당분기 업종별 변화%의 MinMaxScaler를 적용해줍니다")
      print(__name__)
    else:
        print("임포트되어 사용됨")
        print("자동으로 해당분기 업종별 변화%의 MinMaxScaler를 적용해줍니다")
        print(__name__) 
  def rescale(self):
    self.temp = self.temp.replace(np.inf,np.nan)
    self.temp.dropna(axis=0,inplace=True)
    minmax = pd.DataFrame()
    # 업종별 변화추세에 따른 minmaxscaler 적용
    for i in self.df['업종'].unique():
      my_df = self.df[self.df['업종']==i]
      my_df['1분기_변화%'] = self.scaler.fit_transform(my_df['1분기_변화%'].values.reshape(-1,1))
      minmax = pd.concat([minmax,my_df])
    return minmax
  def info(self):
    print("데이터 업종 가짓수",len(self.df['업종'].unique()))
    print("데이터 상권 군집수",len(self.df['상권군집번호'].unique())) # 1분기_변화%가 무한으로 나오는걸 지워서 군집도 다소 날아감
    print("데이터 전체수",len(self.df))

In [211]:
hallo = 분기변화율_스케일조정(분기=1,data=df)
hallo.rescale()

자동으로 해당분기 업종별 변화%의 MinMaxScaler를 적용해줍니다
__main__


,상권군집번호,업종,1분기_변화%
0,0,SB042,0.052110
10,1,SB042,0.393536
22,3,SB042,0.000000
43,7,SB042,0.071006
56,10,SB042,0.074570
...,...,...,...
1287,286,SB002,0.216622
1311,291,SB002,0.307908
1317,294,SB002,0.302788
1333,298,SB002,0.432874


In [76]:
df = pd.read_excel(file_name[-4]).drop(['Unnamed: 0'],axis=1)
df['1분기_변화%'] = df['1분기_변화%'].replace(np.inf,np.nan)
df

,상권군집번호,업종,1분기_변화%
0,0,SB042,-0.471337
1,0,SB041,-0.561587
2,0,SB001,-0.191868
3,0,SB047,-0.352054
4,0,SB029,-0.036802
...,...,...,...
1336,299,SB047,-0.889685
1337,299,SB029,-0.399060
1338,299,SB008,0.020992
1339,299,SB003,-0.254821


### 연관규칙

- support(지지도)

전체 거래에서 특정 물품 A와 B가 동시에 거래되는 비중

해당 규칙이 얼마나 의미있는지 보여줌.

 

지지도 = P(A∩B)

:A와 B가 동시에 일어난 횟수 / 전체 거래 횟수

 

 

- confiddence(신뢰도)

A를 포함하는 거래 중 A와 B가 동시에 거래되는 비중

 

신뢰도 = P(A∩B) / P(A)

:A와 B가 동시에 일어난 횟수 / A가 일어난 횟수

 

 

- lift(향상도)

A라는 상품에서 신뢰도가 동일한 상품 B와 C가 존재할 때, 어떤 상품을 더 추천해야 좋을지 판단.

A와 B가 동시에 거래된 비중을 A와 B가 서로 독립된 사건일 때 동시에 거래된 비중으로 나눈 값

 

향상도 = P(A∩B) / P(A)*P(B) = P (B|A) / P (B)

: A와 B가 동시에 일어난 횟수 / A, B가 독립된 사건일 때 A,B가 동시에 일어날 확률

In [32]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [43]:
te = TransactionEncoder()
te_result = te.fit(tot_list).transform(tot_list)
te_result

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ..., False,  True,  True],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True, False,  True]])

In [48]:
itemset = apriori(pd.DataFrame(te_result), min_support=0.30, use_colnames=True)
itemset

,support,itemsets
0,0.962838,(0)
1,0.479730,(1)
2,0.418919,(2)
3,0.814189,(3)
4,0.469595,(4)
5,0.354730,(8)
6,0.645270,(9)
7,0.469595,"(0, 1)"
8,0.412162,"(0, 2)"
9,0.790541,"(0, 3)"


In [52]:
association_rules(itemset, metric="lift", min_threshold=1) # 향상도가 최소 1이상인 연관관계 출력

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(0),(1),0.962838,0.479730,0.469595,0.487719,1.016654,0.007693,1.015596
1,(1),(0),0.479730,0.962838,0.469595,0.978873,1.016654,0.007693,1.759009
2,(0),(2),0.962838,0.418919,0.412162,0.428070,1.021845,0.008811,1.016001
3,(2),(0),0.418919,0.962838,0.412162,0.983871,1.021845,0.008811,2.304054
4,(0),(3),0.962838,0.814189,0.790541,0.821053,1.008430,0.006608,1.038355
...,...,...,...,...,...,...,...,...,...
63,"(0, 4)",(9),0.456081,0.645270,0.320946,0.703704,1.090557,0.026650,1.197213
64,"(9, 4)",(0),0.331081,0.962838,0.320946,0.969388,1.006803,0.002169,1.213964
65,(0),"(9, 4)",0.962838,0.331081,0.320946,0.333333,1.006803,0.002169,1.003378
66,(9),"(0, 4)",0.645270,0.456081,0.320946,0.497382,1.090557,0.026650,1.082172
